In [1]:
from pathlib import Path
import yaml
import os

root_dir = Path("/media") / "sharedData" / "data"
root_name = "glass-a2744"

catalogue_dir = root_dir / "2024_08_16_A2744_v4" / "glass_niriss" / "match_catalogues"

grizli_home_dir = root_dir / "2024_08_16_A2744_v4" / "grizli_home"
grizli_extraction_dir = grizli_home_dir / "Extractions_v2"

In [2]:
from astropy.table import Table, join, Column, vstack
import numpy as np

v2_out_dir = grizli_home_dir / "classification-stage-2"/"catalogues" / "compiled"
cat_others = Table.read(v2_out_dir  / "internal_stage_3_2.fits")

stage_3_dir = grizli_home_dir / "Extractions_v3"
stage_3_cat = Table.read(stage_3_dir / "classification" / "stage_3_PJW_class.fits")

force_dir = grizli_home_dir / "ForcedExtractions"
detect_cat = Table.read(force_dir / f"{root_name}-ir.cat.fits")

cat_others
# stage_3_cat

NUMBER,NPIX,XMIN,XMAX,YMIN,YMAX,X,Y,X2_IMAGE,Y2_IMAGE,XY_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,FLUX,PEAK,XPEAK,YPEAK,CFLUX,CPEAK,XCPEAK,YCPEAK,X_IMAGE,Y_IMAGE,RA,DEC,X_WORLD,Y_WORLD,FLUX_ISO,FLUXERR_ISO,AREA_ISO,MAG_ISO,KRON_RADIUS,KRON_RCIRC,FLUX_AUTO,FLUXERR_AUTO,BKG_AUTO,FLAG_AUTO,AREA_AUTO,FLUX_RADIUS_FLAG,FLUX_RADIUS_20,FLUX_RADIUS,FLUX_RADIUS_90,TOT_CORR,MAG_AUTO,MAGERR_AUTO,FLUX_APER_0,FLUXERR_APER_0,FLAG_APER_0,BKG_APER_0,MASK_APER_0,FLUX_APER_1,FLUXERR_APER_1,FLAG_APER_1,BKG_APER_1,MASK_APER_1,FLUX_APER_2,FLUXERR_APER_2,FLAG_APER_2,BKG_APER_2,MASK_APER_2,FLUX_APER_3,FLUXERR_APER_3,FLAG_APER_3,BKG_APER_3,MASK_APER_3,FLUX_APER_4,FLUXERR_APER_4,FLAG_APER_4,BKG_APER_4,MASK_APER_4,FLUX_APER_5,FLUXERR_APER_5,FLAG_APER_5,BKG_APER_5,MASK_APER_5,FLUX_APER_6,FLUXERR_APER_6,FLAG_APER_6,BKG_APER_6,MASK_APER_6,id_astrodeep,ra,dec,zspec,EAzY_eazy_v13,EAzY_Larson,EAzY_LarsonLyaRed,flag,id_alt,n_lines_detected,id_astrodeep_zphot,zphot,ID,SEG_ID,"F115W,72.0_COVERAGE","F150W,72.0_COVERAGE","F200W,72.0_COVERAGE","F115W,341.0_COVERAGE","F150W,341.0_COVERAGE","F200W,341.0_COVERAGE",GRIZLI_REDSHIFT,ESTIMATED_REDSHIFT,UNRELIABLE_REDSHIFT,TENTATIVE_REDSHIFT,BAD_SEG_MAP,COMMENTS,V1_CLASS,REDSHIFT_USE,"F115W,72.0_QUALITY_Benedetta","F150W,72.0_QUALITY_Benedetta","F200W,72.0_QUALITY_Benedetta","F115W,341.0_QUALITY_Benedetta","F150W,341.0_QUALITY_Benedetta","F200W,341.0_QUALITY_Benedetta",COMMENTS_Benedetta,ESTIMATED_REDSHIFT_Benedetta,Z_FLAG_Benedetta,"F115W,72.0_QUALITY_Guido","F150W,72.0_QUALITY_Guido","F200W,72.0_QUALITY_Guido","F115W,341.0_QUALITY_Guido","F150W,341.0_QUALITY_Guido","F200W,341.0_QUALITY_Guido",COMMENTS_Guido,ESTIMATED_REDSHIFT_Guido,Z_FLAG_Guido,"F115W,72.0_QUALITY_Matt","F150W,72.0_QUALITY_Matt","F200W,72.0_QUALITY_Matt","F115W,341.0_QUALITY_Matt","F150W,341.0_QUALITY_Matt","F200W,341.0_QUALITY_Matt",COMMENTS_Matt,ESTIMATED_REDSHIFT_Matt,Z_FLAG_Matt,"F115W,72.0_QUALITY_Nicolo","F150W,72.0_QUALITY_Nicolo","F200W,72.0_QUALITY_Nicolo","F115W,341.0_QUALITY_Nicolo","F150W,341.0_QUALITY_Nicolo","F200W,341.0_QUALITY_Nicolo",COMMENTS_Nicolo,ESTIMATED_REDSHIFT_Nicolo,Z_FLAG_Nicolo,"F115W,72.0_QUALITY_Peter","F150W,72.0_QUALITY_Peter","F200W,72.0_QUALITY_Peter","F115W,341.0_QUALITY_Peter","F150W,341.0_QUALITY_Peter","F200W,341.0_QUALITY_Peter",COMMENTS_Peter,ESTIMATED_REDSHIFT_Peter,Z_FLAG_Peter,"F115W,72.0_QUALITY_Sofia","F150W,72.0_QUALITY_Sofia","F200W,72.0_QUALITY_Sofia","F115W,341.0_QUALITY_Sofia","F150W,341.0_QUALITY_Sofia","F200W,341.0_QUALITY_Sofia",COMMENTS_Sofia,ESTIMATED_REDSHIFT_Sofia,Z_FLAG_Sofia,"F115W,72.0_QUALITY_Tommaso","F150W,72.0_QUALITY_Tommaso","F200W,72.0_QUALITY_Tommaso","F115W,341.0_QUALITY_Tommaso","F150W,341.0_QUALITY_Tommaso","F200W,341.0_QUALITY_Tommaso",COMMENTS_Tommaso,ESTIMATED_REDSHIFT_Tommaso,Z_FLAG_Tommaso,"F115W,72.0_QUALITY_Xianlong","F150W,72.0_QUALITY_Xianlong","F200W,72.0_QUALITY_Xianlong","F115W,341.0_QUALITY_Xianlong","F150W,341.0_QUALITY_Xianlong","F200W,341.0_QUALITY_Xianlong",COMMENTS_Xianlong,ESTIMATED_REDSHIFT_Xianlong,Z_FLAG_Xianlong,"F115W,72.0_QUALITY_Yechi","F150W,72.0_QUALITY_Yechi","F200W,72.0_QUALITY_Yechi","F115W,341.0_QUALITY_Yechi","F150W,341.0_QUALITY_Yechi","F200W,341.0_QUALITY_Yechi",COMMENTS_Yechi,ESTIMATED_REDSHIFT_Yechi,Z_FLAG_Yechi,Z_VALS,Z_FLAGS,Z_FLAG_ALL,Z_EST_ALL,N_CLASS,COMMENTS_PJW,Z_FLAG_V3,Z_EST_V3
,pix2,,,,,,,pix2,pix2,pix2,pix,pix,deg,1 / pix2,1 / pix2,1 / pix2,uJy,uJy,,,uJy,uJy,,,,,deg,deg,deg,deg,uJy,uJy,,uJy,pix,pix2,uJy,uJy,uJy,,pix,,,,,,uJy,uJy,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float6

In [3]:
from itertools import product
from numpy.lib.recfunctions import structured_to_unstructured

filters = ["F115W","F150W","F200W"]
pas = ["72.0","341.0"]

beam_names = [f"{a},{b}" for b, a in product(pas, filters)]
coverage_names = [f"{b}_COVERAGE" for b in beam_names]
quality_names = [f"{b}_QUALITY" for b in beam_names]
# new_names = [f"{b}_USE" for b in beam_names]

In [4]:
from numpy.typing import ArrayLike

def table_to_array(table: Table) -> ArrayLike:
    """
    Convert an astropy table to a numpy array.

    Parameters
    ----------
    table : Table
        Original astropy table.

    Returns
    -------
    ArrayLike
        Numpy array.
    """
    return np.lib.recfunctions.structured_to_unstructured(table.as_array()) 

In [5]:
unreliable = [
    2
]

In [6]:
column_names = [
    "ID",
    "RA","DEC","Z_NIRISS","Z_FLAG"
]
column_names.extend(quality_names)
column_names.extend(["Z_OUTSIDE", "Z_ON_EDGE", "Z_REFIT","DZ_1PZ"])

quality_mapping = {
    "Unusable":0,
    "Poor":1,
    "Good":2,
    "Excellent":3
}

# quality_names_others = [c for c in cat_others.colnames if any(q in c for q in quality_names)]
# print (quality_names_others)

In [7]:
# rows_to_use = (np.isin(stage_3_z_cat["NUMBER"],previous_ids) | (stage_3_z_cat["NUMBER"]>=3493)) & ~(stage_3_z_cat["NUMBER"].mask)

In [8]:
table_data = []

match_precision = 3

for row in detect_cat[:]:
    row_data = []
    stage_3_idx = np.argwhere(row["NUMBER"]==stage_3_cat["ID"].astype(int))
    other_idx = np.argwhere(row["NUMBER"]==cat_others["NUMBER"].astype(int))
    if stage_3_idx.size > 0:
        stage_3_row = stage_3_cat[stage_3_idx]
        if "rm" in stage_3_row["COMMENTS"].data[0,0].astype(str):
            print (f"Removing object {row["NUMBER"]}")
            continue
        # print (row["ID"])
        row_data.extend(*table_to_array(stage_3_row["ID","RA","DEC"]))
        z_d = [np.nan, 0]
        if stage_3_row["UNRELIABLE_REDSHIFT"]:
            z_d = [np.nan, 2]
        elif stage_3_row["TENTATIVE_REDSHIFT"]:
            z_d = [stage_3_row["ESTIMATED_REDSHIFT"][0][0],3]
        else:
            z_d = [stage_3_row["ESTIMATED_REDSHIFT"][0][0],4]
        # print (z_d)
        row_data.extend(z_d)
        # row_data.extend(*table_to_array(stage_3_row[coverage_names]))
        # print (row_data)
        # print (table_to_array(stage_3_row[duality_names]))
        row_data.extend([quality_mapping[q] for q in table_to_array(stage_3_row[quality_names])[0].astype(str)])
        # print (map(quality_mapping.get, row_data[quality_names].data))
        # print (stage_3_row["GRIZLI_REDSHIFT","ESTIMATED_REDSHIFT"])
        # try:
        if z_d[1] >=3:
            z_cat = Table.read(stage_3_dir / "full" / f"{root_name}_{int(stage_3_row["ID"][0][0]):0>5}.full.fits", "ZFIT_STACK")
            if (stage_3_row["ESTIMATED_REDSHIFT"]==np.nanmin(z_cat["zgrid"])) or (stage_3_row["ESTIMATED_REDSHIFT"]==np.nanmax(z_cat["zgrid"])):
                # print ("edge", stage_3_row["ID"])
                row_data.extend([False,True])
            elif (stage_3_row["ESTIMATED_REDSHIFT"]>np.nanmin(z_cat["zgrid"])) and (stage_3_row["ESTIMATED_REDSHIFT"]<np.nanmax(z_cat["zgrid"])):
                # print ("contained", stage_3_row["ID"][0])
                row_data.extend([False,False])
            elif (stage_3_row["ESTIMATED_REDSHIFT"]<np.nanmin(z_cat["zgrid"])) or (stage_3_row["ESTIMATED_REDSHIFT"]>np.nanmax(z_cat["zgrid"])):
                # print ("outside", stage_3_row["ID"][0])
                row_data.extend([True,False])

            if np.round(z_cat.meta["Z_MAP"],decimals=match_precision)==np.round(stage_3_row["ESTIMATED_REDSHIFT"][0][0], decimals=match_precision):
                row_data.extend([False,0.])
            else:
                # print (, stage_3_row["ESTIMATED_REDSHIFT"][0][0], "s3",stage_3_row["ID"][0][0])
                row_data.extend([True,(z_cat.meta["Z_MAP"]-stage_3_row["ESTIMATED_REDSHIFT"][0][0])/(1+z_cat.meta["Z_MAP"])])
        else:
            row_data.extend([False,False,False,0.])

        table_data.append(row_data)
    
    elif other_idx.size > 0:
        others_row = cat_others[other_idx]
        # print ()
        if others_row["NUMBER"].astype(int) in unreliable:
            others_row["Z_EST_V3"] = np.nan
            others_row["Z_FLAG_V3"] = 2

        row_data.extend(*table_to_array(others_row["NUMBER","RA","DEC","Z_EST_V3","Z_FLAG_V3"]))
        # row_data.extend(*table_to_array(stage_3_row[coverage_names]))
        # print (row_data)
        # print (table_to_array(stage_3_row[quality_names]))
        for q, c in zip(quality_names, coverage_names):
            # print (q,c)
            if not others_row[c] > 0.1:
                row_data.append(0)
                continue
            # q_names = [s for s in cat_others.colnames if q in s]
            # print (q_names)

            # print ([others_row[n] for n in q_names if not others_row[n].mask])
            q_data = np.nanmedian([int(others_row[n].data.data[0][0]) for n in [s for s in cat_others.colnames if q in s] if not others_row[n].mask])
            
            row_data.append(q_data)
            # print (others_row[q_names[0]].mask)
            # print (table_to_array(others_row[q_names]).astype(str).filled("")[0])
            # print ([quality_mapping[q] for q in table_to_array(others_row[q_names]).astype(str).filled("")[0]])
        # row_data.extend([quality_mapping[q] for q in table_to_array(stage_3_row[quality_names])[0].astype(str)])
        # print (map(quality_mapping.get, row_data[quality_names].data))

        if others_row["Z_FLAG_V3"] >=3:
            try:
                z_cat = Table.read(grizli_home_dir / "Extractions_v2" / "reextracted" / "full" / f"{root_name}_{int(others_row["ID"][0][0]):0>5}.full.fits", "ZFIT_STACK")
            except:
                # try:
                z_cat = Table.read(grizli_home_dir / "Extractions_v2" / "skipped" / "full" / f"{root_name}_{int(others_row["ID"][0][0]):0>5}.full.fits", "ZFIT_STACK")
            if (others_row["ESTIMATED_REDSHIFT"]==np.nanmin(z_cat["zgrid"])) or (others_row["ESTIMATED_REDSHIFT"]==np.nanmax(z_cat["zgrid"])):
                # print ("edge", stage_3_row["ID"])
                row_data.extend([False,True])
            elif (others_row["ESTIMATED_REDSHIFT"]>np.nanmin(z_cat["zgrid"])) and (others_row["ESTIMATED_REDSHIFT"]<np.nanmax(z_cat["zgrid"])):
                # print ("contained", stage_3_row["ID"][0])
                row_data.extend([False,False])
            elif (others_row["ESTIMATED_REDSHIFT"]<np.nanmin(z_cat["zgrid"])) or (others_row["ESTIMATED_REDSHIFT"]>np.nanmax(z_cat["zgrid"])):
                # print ("outside", stage_3_row["ID"][0])
                row_data.extend([True,False])
            
            if np.round(z_cat.meta["Z_MAP"],decimals=match_precision)==np.round(others_row["Z_EST_V3"][0][0], decimals=match_precision):
                row_data.extend([False,0.])
            else:
                row_data.extend([True, (z_cat.meta["Z_MAP"]-others_row["Z_EST_V3"][0][0])/(1+z_cat.meta["Z_MAP"])])
        else:
            row_data.extend([False,False,False, 0.])

        table_data.append(row_data)

# print (column_names)

# print (Table(names = column_names, data = tuple(zip(*table_data))))
print (len(column_names))
print (len(row_data))

stage_4_cat = Table(names = column_names, data = tuple(zip(*table_data)))
stage_4_cat

/media/sharedData/python/py3.12_GLASS_pipeline/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/media/sharedData/python/py3.12_GLASS_pipeline/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Removing object 171
Removing object 206
Removing object 225
Removing object 228
Removing object 260
Removing object 277
Removing object 281
Removing object 291
Removing object 295
Removing object 328
Removing object 334
Removing object 340
Removing object 348
Removing object 374
Removing object 384
Removing object 390
Removing object 409
Removing object 427
Removing object 455
Removing object 541
Removing object 571
Removing object 667
Removing object 678
Removing object 680
Removing object 685
Removing object 757
Removing object 771
Removing object 789
Removing object 795
Removing object 834
Removing object 850
Removing object 934
Removing object 978
Removing object 979
Removing object 1059
Removing object 1090
Removing object 1100
Removing object 1122
Removing object 1127
Removing object 1135
Removing object 1149
Removing object 1153
Removing object 1177
Removing object 1203
Removing object 1209
Removing object 1214
Removing object 1215
Removing object 1217
Removing object 1230
Remov

ID,RA,DEC,Z_NIRISS,Z_FLAG,"F115W,72.0_QUALITY","F150W,72.0_QUALITY","F200W,72.0_QUALITY","F115W,341.0_QUALITY","F150W,341.0_QUALITY","F200W,341.0_QUALITY",Z_OUTSIDE,Z_ON_EDGE,Z_REFIT,DZ_1PZ
bytes32,bytes32,bytes32,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float64
1.0,3.580726811401407,-30.41884945789135,--,1.0,0.0,0.0,0.0,nan,0.0,0.0,False,False,False,0.0
2.0,3.5805950675838516,-30.41840626202294,nan,2.0,0.0,0.0,0.0,2.0,0.0,0.0,False,False,False,0.0
3.0,3.5823409254568075,-30.418327649996915,--,2.0,2.0,1.5,0.0,2.0,1.5,0.0,False,False,False,0.0
4.0,3.5808351518242985,-30.418217634543268,--,2.0,2.0,0.0,0.0,1.5,0.0,0.0,False,False,False,0.0
5.0,3.5841284374988134,-30.417936732145677,--,2.0,2.0,2.0,2.0,2.0,2.0,0.0,False,False,False,0.0
6.0,3.582211382667949,-30.417880981654292,--,2.0,2.0,2.0,1.0,2.0,1.5,0.0,False,False,False,0.0
7.0,3.582027646479966,-30.417712181218253,--,2.0,2.0,2.0,2.0,2.0,2.0,0.0,False,False,False,0.0
8.0,3.582038666896761,-30.416994081787145,--,2.0,2.0,2.0,0.0,2.0,2.0,0.0,False,False,False,0.0
9.0,3.5826286452508738,-30.417417645847184,2.003669475919288,4.0,2.0,2.0,0.0,2.0,0.0,0.0,False,False,False,0.0


In [9]:
stage_4_cat["ID"] = [int(float(s)) for s in stage_4_cat["ID"]]
for c in ["RA","DEC","Z_NIRISS","DZ_1PZ"]:
    stage_4_cat[c] = [float(s) for s in stage_4_cat[c]]
for c in quality_names:
    stage_4_cat[c] = [min(round(s,2),2) if (np.isfinite(s)) else 0 for s in stage_4_cat[c]]
for c in ["Z_FLAG"]:
    stage_4_cat[c] = [int(s) if (np.isfinite(s)) else 0 for s in stage_4_cat[c]]
stage_4_cat

/tmp/ipykernel_106733/279113373.py:3: UserWarning: Warning: converting a masked element to nan.
  stage_4_cat[c] = [float(s) for s in stage_4_cat[c]]


ID,RA,DEC,Z_NIRISS,Z_FLAG,"F115W,72.0_QUALITY","F150W,72.0_QUALITY","F200W,72.0_QUALITY","F115W,341.0_QUALITY","F150W,341.0_QUALITY","F200W,341.0_QUALITY",Z_OUTSIDE,Z_ON_EDGE,Z_REFIT,DZ_1PZ
int64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float64
1,3.580726811401407,-30.41884945789135,nan,1,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,0.0
2,3.5805950675838516,-30.41840626202294,nan,2,0.0,0.0,0.0,2.0,0.0,0.0,False,False,False,0.0
3,3.5823409254568075,-30.418327649996915,nan,2,2.0,1.5,0.0,2.0,1.5,0.0,False,False,False,0.0
4,3.5808351518242985,-30.418217634543268,nan,2,2.0,0.0,0.0,1.5,0.0,0.0,False,False,False,0.0
5,3.5841284374988134,-30.417936732145677,nan,2,2.0,2.0,2.0,2.0,2.0,0.0,False,False,False,0.0
6,3.582211382667949,-30.417880981654292,nan,2,2.0,2.0,1.0,2.0,1.5,0.0,False,False,False,0.0
7,3.582027646479966,-30.417712181218253,nan,2,2.0,2.0,2.0,2.0,2.0,0.0,False,False,False,0.0
8,3.582038666896761,-30.416994081787145,nan,2,2.0,2.0,0.0,2.0,2.0,0.0,False,False,False,0.0
9,3.5826286452508738,-30.417417645847184,2.003669475919288,4,2.0,2.0,0.0,2.0,0.0,0.0,False,False,False,0.0


In [10]:

# Benedetta's high-z galaxy
stage_4_cat["Z_FLAG"][stage_4_cat["ID"]==2992] = 4

# From Boyett+22
stage_4_cat["Z_FLAG"][stage_4_cat["ID"]==1043] = 3
stage_4_cat["Z_NIRISS"][stage_4_cat["ID"]==1043] = 1.342
stage_4_cat["Z_ON_EDGE"][stage_4_cat["ID"]==1043] = True

# From Boyett+22
stage_4_cat["Z_FLAG"][stage_4_cat["ID"]==2964] = 3
stage_4_cat["Z_NIRISS"][stage_4_cat["ID"]==2964] = 1.485
stage_4_cat["Z_ON_EDGE"][stage_4_cat["ID"]==2964] = True

# From Boyett+22
stage_4_cat["Z_FLAG"][stage_4_cat["ID"]==749] = 3
stage_4_cat["Z_NIRISS"][stage_4_cat["ID"]==749] = 1.043
stage_4_cat["Z_ON_EDGE"][stage_4_cat["ID"]==749] = True

stage_4_cat[stage_4_cat["ID"]==3518]
# stage_4_cat[stage_4_cat["ID"]==3518]
stage_4_cat[stage_4_cat["ID"]==749]

# 1043 1.342
# 2694 1.485
# 749  1.043


ID,RA,DEC,Z_NIRISS,Z_FLAG,"F115W,72.0_QUALITY","F150W,72.0_QUALITY","F200W,72.0_QUALITY","F115W,341.0_QUALITY","F150W,341.0_QUALITY","F200W,341.0_QUALITY",Z_OUTSIDE,Z_ON_EDGE,Z_REFIT,DZ_1PZ
int64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float64
749,3.595889370351209,-30.4044265434742,1.043,3,0.0,2.0,2.0,2.0,1.0,0.0,False,True,False,0.0


In [11]:
from astropy.coordinates import SkyCoord, match_coordinates_sky

stage_4_coords = SkyCoord(ra=stage_4_cat["RA"], dec=stage_4_cat["DEC"], unit="deg")

high_z_data = np.array(
    [
        ["YD4", 3.6038544, -30.3822365, 7.8758],
        ["YD7", 3.6033909, -30.3822289, 7.8772],
        ["ZD6", 3.6065702, -30.3808918, 7.8843],
        ["YD8", 3.5960841, -30.3858051, 7.8869],
        ["ZD2", 3.6045184, -30.3804321, 7.8800],
        ["ZD3", 3.6064637, -30.3809624, 7.8816],
        ["GLASSz8-2", 3.6013467, -30.3791904, 7.8831],
        ["GLASSz8-1", 3.60452, -30.38047, 8.04],
        ["JD1", 3.5950014, -30.4007411, 9.793],
    ]
)
high_z_prev = Table(names=["ID", "RA", "DEC", "Z"], data=high_z_data)
# # Roberts-Borsani+22
# high_z_prev.add_row([3.60451, -30.38046, 8.04])
# high_z_prev.add_row([3.60135, -30.37921, 7.90])


high_z_coords = SkyCoord(ra=high_z_prev["RA"], dec=high_z_prev["DEC"], unit="deg")
# high_z_prev["RA"] = [3.60451,3.60135]
# high_z_prev["DEC"] = [3.60451,3.60135]
#     data=[
#         [1,2,3],
#         [1,3,4]
#     ]
# )
# high_z_prev
idxs, sep, _ = match_coordinates_sky(high_z_coords, stage_4_coords)

for row, idx in zip(high_z_data,idxs):
    print (np.asarray(stage_4_cat["ID","Z_NIRISS"][idx]), row)



(2913, nan) ['YD4' '3.6038544' '-30.3822365' '7.8758']
(2921, nan) ['YD7' '3.6033909' '-30.3822289' '7.8772']
(3013, 8.19689055) ['ZD6' '3.6065702' '-30.3808918' '7.8843']
(2514, 8.05571033) ['YD8' '3.5960841' '-30.3858051' '7.8869']
(3062, 7.875) ['ZD2' '3.6045184' '-30.3804321' '7.88']
(3009, 8.16260922) ['ZD3' '3.6064637' '-30.3809624' '7.8816']
(3162, 7.878) ['GLASSz8-2' '3.6013467' '-30.3791904' '7.8831']
(3062, 7.875) ['GLASSz8-1' '3.60452' '-30.38047' '8.04']
(1121, nan) ['JD1' '3.5950014' '-30.4007411' '9.793']


In [12]:
from astropy.coordinates import SkyCoord, match_coordinates_sky

stage_4_coords = SkyCoord(ra=stage_4_cat["RA"], dec=stage_4_cat["DEC"], unit="deg")

high_z_data = np.array(
    [
        ["YD4", 3.6038544, -30.3822365, 7.8758],
        ["YD7", 3.6033909, -30.3822289, 7.8772],
        ["ZD6", 3.6065702, -30.3808918, 7.8843],
        ["YD8", 3.5960841, -30.3858051, 7.8869],
        ["ZD2", 3.6045184, -30.3804321, 7.8800],
        ["ZD3", 3.6064637, -30.3809624, 7.8816],
        ["GLASSz8-2", 3.6013467, -30.3791904, 7.8831],
        ["GLASSz8-1", 3.60452, -30.38047, 8.04],
        ["JD1", 3.5950014, -30.4007411, 9.793],
    ]
)
high_z_prev = Table(names=["ID", "RA", "DEC", "Z"], data=high_z_data)
# # Roberts-Borsani+22
# high_z_prev.add_row([3.60451, -30.38046, 8.04])
# high_z_prev.add_row([3.60135, -30.37921, 7.90])


high_z_coords = SkyCoord(ra=high_z_prev["RA"], dec=high_z_prev["DEC"], unit="deg")
# high_z_prev["RA"] = [3.60451,3.60135]
# high_z_prev["DEC"] = [3.60451,3.60135]
#     data=[
#         [1,2,3],
#         [1,3,4]
#     ]
# )
# high_z_prev
idxs, sep, _ = match_coordinates_sky(high_z_coords, stage_4_coords)

for row, idx in zip(high_z_data,idxs):
    print (np.asarray(stage_4_cat["ID","Z_NIRISS"][idx]), row)



(2913, nan) ['YD4' '3.6038544' '-30.3822365' '7.8758']
(2921, nan) ['YD7' '3.6033909' '-30.3822289' '7.8772']
(3013, 8.19689055) ['ZD6' '3.6065702' '-30.3808918' '7.8843']
(2514, 8.05571033) ['YD8' '3.5960841' '-30.3858051' '7.8869']
(3062, 7.875) ['ZD2' '3.6045184' '-30.3804321' '7.88']
(3009, 8.16260922) ['ZD3' '3.6064637' '-30.3809624' '7.8816']
(3162, 7.878) ['GLASSz8-2' '3.6013467' '-30.3791904' '7.8831']
(3062, 7.875) ['GLASSz8-1' '3.60452' '-30.38047' '8.04']
(1121, nan) ['JD1' '3.5950014' '-30.4007411' '9.793']


In [13]:
import astropy.units as u
# boyett+22
boyett_tab = Table.read(catalogue_dir / "Boyett+22.fits")

boyett_coords = SkyCoord(ra=boyett_tab["RAdeg"], dec=boyett_tab["DEdeg"], unit="deg")

idxs, sep, _ = match_coordinates_sky(boyett_coords, stage_4_coords)

for row, idx, s in zip(boyett_tab,idxs, sep):
    if s>0.5*u.arcsec:
        continue 
    # print (row["z"],stage_4_cat["Z_NIRISS"][idx])
    glass = stage_4_cat[idx]
    dz = (row["z"]-glass["Z_NIRISS"])/(1+row["z"])
    if np.abs(dz)>1e-3:
        if np.abs(dz)>0.02:
            print ("!!")
        print (f"Boyett: {row["ID"]: >5}, z={row["z"]:.3f}, GLASS: {glass["ID"]: >5}, z={glass["Z_NIRISS"]:.3f}")
    if not np.isfinite(dz):
        if np.abs(dz)>0.02:
            print ("!!")
        print (f"Boyett: {row["ID"]: >5}, z={row["z"]:.3f}, GLASS: {glass["ID"]: >5}, z={glass["Z_NIRISS"]:.3f}, sep={s.to(u.arcsec):.3f}")
        #, np.array(row["z"]), , s.to(u.arcsec), np.array(list(,"Z_NIRISS"][idx])))


# 749
# 3518
# 3166
# 3501
# 2599

# 1043 1.342
# 2964 1.485
# 749  1.043


Boyett:   328, z=1.038, GLASS:  3518, z=nan, sep=0.093 arcsec
Boyett:  2081, z=1.039, GLASS:  2216, z=1.041
Boyett:  2243, z=1.039, GLASS:  2373, z=1.042
Boyett:  2201, z=1.051, GLASS:  2346, z=1.045
Boyett:  2872, z=1.098, GLASS:  3166, z=nan, sep=0.076 arcsec
Boyett:   189, z=1.041, GLASS:  3501, z=nan, sep=0.355 arcsec
Boyett:   553, z=1.118, GLASS:   521, z=1.091
Boyett:   442, z=1.099, GLASS:   452, z=1.097
Boyett:   187, z=1.102, GLASS:   197, z=1.097
!!
Boyett:  1738, z=1.098, GLASS:  1776, z=1.243
Boyett:   277, z=1.096, GLASS:   341, z=1.099
Boyett:   699, z=1.104, GLASS:  3609, z=1.101
Boyett:   515, z=1.108, GLASS:   523, z=1.103
Boyett:  1710, z=1.166, GLASS:  1642, z=1.159
Boyett:  2063, z=1.168, GLASS:  2166, z=1.163
Boyett:  1972, z=1.159, GLASS:  2091, z=1.156
Boyett:  1761, z=1.170, GLASS:  1823, z=1.167
!!
Boyett:  1532, z=1.125, GLASS:  1511, z=2.645
Boyett:  2352, z=1.227, GLASS:  2599, z=nan, sep=0.091 arcsec
Boyett:   138, z=1.275, GLASS:   142, z=1.268
Boyett:   

In [14]:
stage_4_cat[325:350]

ID,RA,DEC,Z_NIRISS,Z_FLAG,"F115W,72.0_QUALITY","F150W,72.0_QUALITY","F200W,72.0_QUALITY","F115W,341.0_QUALITY","F150W,341.0_QUALITY","F200W,341.0_QUALITY",Z_OUTSIDE,Z_ON_EDGE,Z_REFIT,DZ_1PZ
int64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float64
341,3.60264402614227,-30.40987146725151,1.0990000000000002,4,2.0,2.0,1.5,2.0,2.0,1.5,False,True,False,0.0
342,3.5846167616456124,-30.410005752428667,nan,2,0.0,0.0,2.0,0.0,0.0,0.0,False,False,False,0.0
343,3.5923098235703956,-30.409872680571237,nan,2,0.5,0.5,0.5,1.5,1.5,1.5,False,False,False,0.0
344,3.592120842829752,-30.409892958519936,nan,2,1.5,1.0,1.0,0.0,0.0,0.0,False,False,False,0.0
345,3.5845813958251,-30.409794628262972,2.6537366511711244,4,2.0,2.0,2.0,2.0,2.0,2.0,False,False,True,0.0010216482424440776
346,3.591409306380343,-30.40992286290728,nan,0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,0.0
347,3.5997710893951793,-30.409764935315167,nan,2,2.0,2.0,2.0,2.0,2.0,2.0,False,False,False,0.0
349,3.60910219106354,-30.40965662282532,nan,2,1.5,0.0,1.5,2.0,2.0,2.0,False,False,False,0.0
350,3.5925021421102605,-30.40984030801854,nan,2,1.0,1.0,1.0,0.0,0.0,0.0,False,False,False,0.0


In [15]:
stage_4_dir = grizli_home_dir / "Extractions_v4"
(stage_4_dir / "catalogues").mkdir(exist_ok=True, parents=True)


print(np.unique(np.array(stage_4_cat["Z_FLAG"]), return_counts=True))
# print (column_names)
print(np.nansum(stage_4_cat["Z_ON_EDGE"]))
print(np.nansum(stage_4_cat["Z_REFIT"]))
print(
    np.nansum(
        stage_4_cat["Z_REFIT"]
        & (~stage_4_cat["Z_OUTSIDE"])
        & (np.abs(stage_4_cat["DZ_1PZ"]) > 0.001)
    )
)
print(np.nansum(stage_4_cat["Z_REFIT"] & stage_4_cat["Z_OUTSIDE"]))
# with np.printoptions(threshold=np.inf):
#     for s in stage_4_cat[stage_4_cat["Z_REFIT"] & (~stage_4_cat["Z_OUTSIDE"]) & (np.abs(stage_4_cat["DZ_1PZ"])>0.001)]["ID","Z_NIRISS","DZ_1PZ"]:
#         print (list(s))

mag_cat = detect_cat["NUMBER","MAG_AUTO"]

stage_4_cat = join(stage_4_cat, mag_cat, keys_left=["ID"], keys_right=["NUMBER"])


stage_4_cat.write( stage_4_dir / "catalogues" / "stage_4_input_fixed_precision.fits")


(array([0, 1, 2, 3, 4]), array([ 511,  907, 1625,  133,  478]))
93
133
69
3
